# Demo

After looking into some records in the data, I decide to decompose the procedure of finding the unified title for each raw title into the following steps:

1. Load the data into `pandas.Dataframe` object and sort all raw titles alphabetically
2. Clean the raw title
3. Find the root word of each raw title
4. Compute the longest common substring between titles with the same root word
5. Use the shortest longest common substring as unified title 
6. Recover the original index
7. Write the new table with unified titles to an Excel file

**Note**: Step 1,5,6,7 are wrapped together in the `main()` function which will be demonstrated in the last


## Step 1. Load the data into pandas.Dataframe and sort all raw titles alphabetically

Firsly, I load the data into `pandas.Dataframe` object in the standard way. Then I keep the original index in the column `index` and reset the index of dataframe after sorting the `raw_title` column alphabetically. 

**Note**: This is a porition of the code in the `main()` function

```python
# Load the data
data = pd.read_excel('raw_titles.xlsx')

# Sort the dataframe by raw titles alphabetically
# Titles with same root word are put next to each other
data.sort_values('raw_title', inplace=True)
# Reset the index for dataframe and keep the original index
data.reset_index(inplace = True)
```

## Step 2. Clean the raw title

### A. Replace punctuations within the title, except period, with whitespace

Many raw titles contain punctuation characters which do not have useful meanings. And these punctuation characters prevent the longest common substring algorithm from finding the optimal result. Few examples below can demonstrate the influence:

- `BELL & GOSSETT` and `BELL GOSSETT` have longest common substring of `BELL` rather than `BELL GOSSETT`
- `DBI SALA` and `DBI/SALA` have longest common substring of `DBI` rather than `DBI SALA`

Therefore, I decide to remove all punctuations from the raw title except period(.) because period is used a lot in abbreviation, which makes it important to remain in the title

```python
def replace_punctuations(s, delimiter = ' '):
    '''
    Replace every punctuation character in the input string with delimiter
    
    Parameters
    ----------
    s: str
        Input string
    delimiter: str
        Delimiter used to replace punctuation character (the default is ' ')
        
    Returns
    -------
    new_s: str
        Return the string where every original punctuation character is replaced with delimiter
        
    Raises
    ------
    TypeError
        If the input is not a string
    '''
    
    # Validate the input variable type
    if not isinstance(s, str):
        raise TypeError("Input must be a string")
    if not isinstance(delimiter, str):
        raise TypeError("Delimiter must be a string")
        
    # Get all punctuations except period(.), which may be used in acronyms
    punctuations = ''.join([punct for punct in string.punctuation if punct != '.'])
    # Create the regex format to capture any punctuation characters
    regex_punct = r'['+re.escape(punctuations)+']'
    # Replace every punctuation character in the input string with a single space
    new_s = re.sub(regex_punct, delimiter, s)
    # Remove redundant spaces
    new_s = re.sub(r''+re.escape(delimiter)+'+',' ', new_s)
    # Return
    return new_s
```

To implement the function `replace_punctuations(s, delimiter = ' ')`, I used the `punctuation` from `string` module, which contains ``!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~``. 

The default delimiter is a single whitespace for this task, which can be changed in other scenarios given different format of raw titles

Since the delimiter is whitespace, every punctuation in the title is replaced with whitespace also.

Before returning the new title, multiple whitespaces are replaced with a single whitespace.

In [35]:
# Demo for replace_punctuations()
print('After replacing punctuations from the \'{}\':\n{}'
      .format('BELL & GOSSETT', replace_punctuations('BELL & GOSSETT')))

After replacing punctuations from the 'BELL & GOSSETT':
BELL GOSSETT


### B. Remove meaningless words from the title

Many titles contain abbreviations for the form of the business enterprise. In the given sample unified titles, it appears that those abbreviations are not desired to remain in the unified titles:

<br>

|raw_title                         |unified_title|
|:---------:                       |:-------------:|
| MEDIA LUNA PRODUCTIONS LLC       | MEDIA LUNA
| N SYSTEMS INC                    | N SYSTEMS 
| POWER- LINK SOFTWARE SYSTEMS INC | POWER- LINK SOFTWARE SYSTEMS

Therefore, I decide to remove any noticeable abbreviations for business type.

But other words that are meaningless and not desired to be kept in the title can be added in the future

```python
def remove_words(title, wordList):
    '''
    Remove meaningless words from the title. 
    Mainly abbreviations for the form of business enterprise.
    
    - Coporations:
        - Corp: Corporation
        - INC: Incorporated Company
        - PC: Professional Coporation
        - SC: Service Coporation
    - Partnerships:
        - GP: Genearl Partnership
        - LP: Limited Partnership
        - LLP: Limited Liability Partnership
        - LLLP: Limited Liability Limied Partnership
    - Limited Liability Companies:
        - LLC: Limited Liability Company
        - LC: Limited Liability Company
        - LTD.: Limited Company
        - PLLC: Professional Limited Liability Company
    
    Parameters
    ----------
    title:str
        Input title
    wordList:[str]
        A list of words to remove from the title
    
    Returns
    -------
    clean_title: str
         Cleaned title after words removal
    
    Raises
    ------
    TypeError
        If the input word list for removal are not a list of strings
    '''
    # Validate input variable type
    if not isinstance(wordList, list):
        raise TypeError('The input word list for removal must be a list')
    if not all([isinstance(word, str) for word in wordList]):
        raise TypeError('Every word in the list must be in string format')    
    
    # Remove words from the title if it appears in the given word list for removal
    clean_title = [word for word in title.split() if word not in wordList]
    # Combine the rest of words together in a string separated by whitespace
    clean_title = ' '.join(clean_title)
    
    # Return the cleaned title
    return clean_title
```

In [36]:
# Demo for remove_words()
print('After words removal from \'{}\':\n{} '
      .format('POWER- LINK SOFTWARE SYSTEMS INC',
              remove_words(title = 'POWER- LINK SOFTWARE SYSTEMS INC',
                           wordList = ['INC','PC','SC','GP','LP','LLP','CORP'
                                       'LLLP','LLC','LC','LTD','PLLC'])))

After words removal from 'POWER- LINK SOFTWARE SYSTEMS INC':
POWER- LINK SOFTWARE SYSTEMS 


### C. Finish title cleaning

To wrap both **Step A** and **Step B** in a single function, I defined `clean_title()`. 

In addition to calling `replace_punctuations()` and `remove_words()` on input title, `clean_title()` also check if the remaining title is an empty string after the cleaning. If the answer is yes, it means the current logic of title cleaning is improper for it. The solution is to skip the cleaning procedure for such titles by restoring them to the original raw titles.

One example of such raw titles is `LC CORP` at 720th row in the Excel file. This title is consisted entirely by abbreviations of business types by accident

```python
def clean_title(title, wordList):
    '''
    Clean the input title and return the new one
    
    Parameters
    ----------
    wordList: [str]
        The list of words targeted to be removed from titles
        
    Returns
    -------
    new_title: str
        The cleaned title
    '''
    
    # Replace puncatuations in the title with whitespace
    new_title = replace_punctuations(title)
    # Remove words in the given list from the title
    new_title = remove_words(new_title, wordList)
    
    # If after the cleaning, nothing is left
    if new_title == '':
        # Skip the cleaning procedure and restore the original raw title
        new_title = title
    
    # Return the cleaned title
    return new_title
```

## Step 3. Find the root word of each raw title

I made an important assumption when performing the title unification in this task: **Only titles that have the same root word should be unified**. To define what is a **root word**, I declare two requirements:

- The word is at the beginning of the title
- The word is consisted of more than one character

Any continuous sequence of characters within the title that meets the above two requirements will be considered as the **root word** for the title

The second requirement is needed here because some titles have the same first word, which only has one character, but are apparently not supposed to be unified since the rest characters are not similar at all

| raw_title |
|:----------:|
| J & B SOFTWARE INC |
| J & C ASSOCIATES I LLC |
| J & N RECORDS LLC |
| A - 1 FARGO VAN AND STORAGE INC |
| A AND E TELEVISION NETWORKS LLC |
| ... |

Furthermore, I also see some titles that start with the word `THE`. By the definition above, `THE` will be the root word for any titles that start with it, which obviously does not make sense.

| raw_title |
|:----------:|
| THE ANGLERS RESORT |
| THE BEACH CLUB |
| THE BOX |
| THE CHENILLE KRAFT COMPAN |
| ... |

Therefore, I added an optional parameter `ignore` in the function, which contains a list of words to ignore when trying to locate the root word of a title

```python
def root_word(title, ignore):
    '''
    Obtain the root word of a title based on the following assumptions:
    (1) The root word is at the beginning of the title
    (2) The root word is consisted of more than one character
    
    Parameters
    ----------
    title: str
        Input title with space as delimiter
    ignore: [str]
        The list of words to ignore when locating the root word of a title
        
    Returns
    -------
    root: str
        The root word of the input title
    
    Raises
    ------
    TypeError
        If the input title is not a string
    '''
    
    # Obtain the list of words in the title
    words = title.split()
    
    # Remove the first word from consideration if it is among the ignore words
    if words[0] in ignore:
        words.pop(0)
    
    # Assume the first word is root word
    root = words.pop(0)
    
    # If the first word has only one character
    if len(root) == 1:
        # Loop through the rest words in the list in order one by one, 
        # stop when reach a word that has more than one characters
        while len(words[0]) <= 1:
            # Pop out the first word from the list
            # Add the first word to root word with space delimiter
            root += ' ' + words.pop(0)

    # Return the root word
    return root
```

In [38]:
# Demo for root_word()
print('The root word for \'{}\' is:\n{}'.format('ADOBE SYSTEMS', 
                                           root_word('ADOBE SYSTEMS', ignore = ['THE'])))
print('\nThe root word for \'{}\' is:\n{}'.format('THE ANGLERS RESORT', 
                                           root_word('THE ANGLERS RESORT', ignore = ['THE'])))

The root word for 'ADOBE SYSTEMS' is:
ADOBE

The root word for 'THE ANGLERS RESORT' is:
ANGLERS


## Step 4. Compute the longest common substring between titles with the same root word

I transform the problem of determining the unified title into computing the longest common substring, with complete words, between titles with the same root word. For example, two titles below have the same root word `CLARK`

- CLARK FOAM
- CLARK FOAM PRODUCTS CORPORATION

The longest common substring between them is `CLARK FOAM`, which will be used as their unified title

```python
def longest_common_substring(s1, s2):
    '''
    Use dynamic programming to find the longest common substring, with complete words, 
    of two input strings
    
    Parameters
    ----------
    s1: str
        First input string
    s2: str
        Second input string
    
    Returns
    -------
    substring: str
        Return the longest common substring
        
    Raises
    ------
    TypeError
        If the inputs are not strings
    '''
    
    # Validate the input variable type
    if not isinstance(s1, str):
        raise TypeError("First input must be a string")
    if not isinstance(s2, str):
        raise TypeError("Second input must be a string")
    
    # Split the first string into list of words
    s1 = s1.split()
    # Split the second string into list of words
    s2 = s2.split()
    
    # Initialize a matrix M filled with zeroes to store 
    # lengths of the longest common substring
    # M[i][j] contains the lengh of longest common substring between 
    # s1[0,1,...,i-1] and s2[0,1,...,j-1]
    M = np.array([[0] * (1 + len(s2)) for i in range(1 + len(s1))])
    
    # Initialize the lenghth of the longest common substring to zero
    longest_length = 0
    
    # Compare all characters in s1 and all characters in s2 pair by pair
    for i in range(1, 1 + len(s1)):
        for j in range(1, 1 + len(s2)):
            # If i-1 th character in s1 matches j-1 th character in s2
            if s1[i - 1] == s2[j - 1]:
                # Add the longest length by 1
                M[i][j] = M[i - 1][j - 1] + 1
                # Update the longest_length
                longest_length = max(longest_length,M[i][j])
    
    # Find the end position of the longest common substring in s1
    s1_end = np.where(M == longest_length)[0].item(0)
    # Calculate the start position of the the longest common substring in s1
    s1_start = s1_end - longest_length
    # Extract the longest common substring
    substring = ' '.join(s1[s1_start: s1_end])
    
    # Return the longest common substring
    return substring
```

In [43]:
# Demo for longest_common_substring()
longest_common_substring('CLARK FOAM','CLARK FOAM PRODUCTS CORPORATION')

'CLARK FOAM'

## Step 5. Use the shortest longest common substring as unified title 

In order to find the unified title for more than 2 raw titles that are supposed to be unified together. I choose the shortest result from `longest_common_substring()` when it is being called on every possible pairs. To demonstrate, consider the following three raw titles

- PELICAN
- PELICAN PRODUCTS
- PELICAN PRODUCTS-PHOTO VIDEO

By calling `longest_common_substring()` on all possible pairs of these three titles, I will get a list of results `['PELICAN','PELICAN','PELICAN PRODUCTS']`. Then I will use the shortest one from the list, `PELICAN`, as the unified title for all three raw titles

To perform such operation on all raw titles, I defined `unify_title()` to wrap up all necessary operations inside one function. This function make a deep copy of input dataframe, then will loop through all raw titles within a dataframe and insert the correponding unified title into the new dataframe. Finally, `unify_title()` will return a new dataframe with all raw titles and their unified titles


```python
def unify_title(dataframe, raw_column, unified_column, ignore, remove):
    '''
    Unify similar titles in raw_column of dataframe and 
    put the same unified title for all of them in corresponding unified_column
    
    Parameters
    ----------
    dataframe: pandas.core.frame.DataFrame
        Input table containing at least a raw title column which need to be unified
    raw_column: str
        Column name in the dataframe that contains raw titles
    unified_column:str
        Column name in the dataframe that will store the unified titles
    ignore: [str]
        The list of words to ignore when locating the root word of a title
    remove: [str]
        The list of words targeted to be removed from titles
    
    Returns
    -------
    result: pandas.core.frame.DataFrame
        Return a table with both original raw titles and their unified titles
        
    Raises
    ------
    TypeError
        If the input table is not pandas dataframe, or column names are not strings
    '''
    
    # Validate the input variable type
    if not isinstance(dataframe, pd.DataFrame):
        raise TypeError("Input table must be pandas DataFrame")
    if not isinstance(raw_column, str):
        raise TypeError("Column name in the dataframe for raw titles must be a string")
    if not isinstance(unified_column, str):
        raise TypeError("Column name in the dataframe to store unified titles must be a string")
    
    # Make a deep copy of original data
    result = dataframe.copy()
    
    # Obtain the list of raw titles
    raw_title = result[raw_column]
    
    # Initialize i and j
    # i: the index of current raw title
    # j: the index of the raw title to compare with
    i = 0
    j = i
    # Obtain the max value for index j (The number of rows in dataframe)
    _max = result.shape[0]
    
    # While the index j is NOT out of range
    while j < _max:
        # Obtain the current raw title and clean it
        raw = clean_title(raw_title[i], remove)
        # Obtain the root word from the raw_title[i]
        root = root_word(raw, ignore)
        # Create an empty list to store 
        # longest common substring for each pair of raw titles
        common_title_list = []

        # Loop through all raw titles starting from index i
        # Find the longest common substring between raw_title[i] and raw_title[j] 
        # Stop when
        # (1) Reach the last row (j == _max - 1)
        # (2) raw_title[i] and raw_title[j] DO NOT have the same root word 
        #     (Should NOT be unified)
        for j in range(i, _max):
            # Obtain the current raw title and clean it
            new_raw = clean_title(raw_title[j], remove)
            # Obtain the root word for raw_title[j]
            new_root = root_word(new_raw, ignore)
            
            # If the two raw titles have the same root word
            # Try to unify these two titles
            if new_root == root:
                # Find the longest common substring
                common_title = longest_common_substring(raw,new_raw)
                # Add the current longest common substring to the list
                common_title_list.append(common_title)
                # If the index reach the last row
                if j == _max - 1:
                    # Use the shortest substring from the common_title_list
                    # to be the unified title
                    unified_title = min(common_title_list, key = len)
                    # If the unified title does not contain the root word
                    if root not in unified_title:
                        # Discard the longest common substring and
                        # set the root word to be the unified title
                        unified_title = root
                    
                    # Insert the same unified title for all current raw titles
                    result.loc[i:j,unified_column] = unified_title
                    # Return the new dataframe with filled unified_column
                    return result
            
            # If the two raw titles DO NOT have the same root word
            # Finish the current title unification
            else:
                # Use the shortest substring from the common_title_list
                # to be the unified title
                unified_title = min(common_title_list, key = len)
                # If the unified title does not contain the root word
                if root not in unified_title:
                    # Discard the longest common substring and
                    # set the root word to be the unified title
                    unified_title = root
                    
                # Insert the same unified title for all current raw titles
                result.loc[i:j-1,unified_column] = unified_title
                # Reset index i to start title unification for the next raw title
                i = j
                # Jump out of the inner j for loop
                break
```

In [22]:
# Demo for unify_title()

# Create a small dataframe
df = pd.DataFrame({'raw_title':['AMES RESEARCH LABORATOR',
                                'AMES RESEARCH LABORATORIES',
                                'AMES RESEARCH LABORATORIES INC'],
                   'unified_title':'NA'})
# Show the demo dataframe
print('This is the original demo dataframe:')
print('-'*47)
print(df)

print('\n'*2)

# Call unify_title() on the demo dataframe with parameters:
print('This is the returned dataframe after calling unify_title() with parameters:')
print("- ignore = ['THE']")
print("- remove = ['INC']")
print('-'*48)
print(unify_title(df,'raw_title','unified_title',ignore=['THE'],remove=['INC']))

This is the original demo dataframe:
-----------------------------------------------
                        raw_title unified_title
0         AMES RESEARCH LABORATOR            NA
1      AMES RESEARCH LABORATORIES            NA
2  AMES RESEARCH LABORATORIES INC            NA



This is the returned dataframe after calling unify_title() with parameters:
- ignore = ['THE']
- remove = ['INC']
------------------------------------------------
                        raw_title  unified_title
0         AMES RESEARCH LABORATOR  AMES RESEARCH
1      AMES RESEARCH LABORATORIES  AMES RESEARCH
2  AMES RESEARCH LABORATORIES INC  AMES RESEARCH


## Step 6. Recover the original index

To recover the original index, I simply use the standard way as shown below

**Note**: This is a porition of the code in the `main()` function

```python
# Recover the original index
unified_data.set_index('index', inplace=True)
# Sort the dataframe by original index, so it looks the same
unified_data.sort_index(inplace = True)
```

## Step 7. Write the new table with unified titles to an Excel file

I have performed all operations on a `pandas.Dataframe` object, I need to write it out in an Excel file now. I utilized the `pd.ExcelWriter()` function and wrap up few lines of code in a function called `dataframe_to_excel()` to make things clearer in the `main()` function later on.

```python
def dataframe_to_excel(dataframe, filename, sheetname, with_index = False):
    '''
    Write a pandas dataframe to excel
    
    Parameters
    ----------
    dataframe: pandas.core.frame.DataFrame
        Dataframe object that need to be written to excel
    filename: str
        Name of output excel file
    sheetname:
        Name of output sheet in excel
    with_index: bool
        Whether to include dataframe index as a column in excel or not
    
    Raises
    ------
    TypeError
        If input variables are not in correct data type
    '''
    
    # Validate the input variable type
    if not isinstance(dataframe, pd.DataFrame):
        raise TypeError("Input table must be pandas DataFrame")
    if not isinstance(filename, str):
        raise TypeError("Name of output excel file must be a string")
    if not isinstance(sheetname, str):
        raise TypeError("Name of output sheet in excel must be a string")
    if not isinstance(with_index, bool):
        raise TypeError("with_index can only take boolean value")
    
    # Initialize an Excel writier
    writer = pd.ExcelWriter(filename)
    # Write out the unified result
    dataframe.to_excel(writer, sheet_name = sheetname, index = with_index)
    writer.save()
```

## The `main()` function

This is the `main()` function that will be executed. It loads the data from `raw_titles.xlsx` in current directory, performs the unifications for all raw titles, and write out the result to `unified_titles.xlsx`, with sheet name of `unified`, in the current directory 

```python
def main():
    '''
    The main function to perform the title unification including:
    1. Load the data from 'raw_titles.xlsx' in current directory
    2. Perform unifications for titles with the same starting word after cleaning
    3. Write the original raw titles and unified titles together to an excel
    '''
    
    # Load the data
    data = pd.read_excel('raw_titles.xlsx')
    
    # Sort the dataframe by raw titles alphabetically
    # Titles with same root word are put next to each other
    data.sort_values('raw_title', inplace=True)
    # Reset the index for dataframe and keep the original index
    data.reset_index(inplace = True)
    
    # Unify raw titles that start with the same word
    unified_data = unify_title(dataframe = data, 
                               raw_column = 'raw_title', 
                               unified_column = 'unified_title',
                               ignore = ['THE'],
                               remove = ['INC','PC','SC','GP','LP','LLP','CORP'
                                         'LLLP','LLC','LC','LTD','PLLC'])
    
    # Recover the original index
    unified_data.set_index('index', inplace=True)
    # Sort the dataframe by original index, so it looks the same
    unified_data.sort_index(inplace = True)
    
    # Write the dataframe to excel
    dataframe_to_excel(dataframe = unified_data, 
                       filename = 'unified_title.xlsx',
                       sheetname = 'unified', 
                       with_index = False)
```

# Future Improvements

The result is not 100% correct for now just by quickly glancing over it. Below are several things I noticed so far that I think could be improved in the future if I have more time

**(1) The choose of root word**

In my current algorithm, finding the unified title starts with locating the root word of a title. However, my way of locating the root word or even the current definition of root word is not perfect. There are quite a few titles that will be put with incorrect unified title due to this problem:

- C AND C SIGNS AND GRAPHIC DESIGN INC
- C AND H DISPOSAL SERVICE INC
- C AND S SALES
- C MAX MEDIA LLC

None of the above titles should even be unified but my result produces that all of them have unified title of a single character `C`

Therefore, a better definition of root word, or the way of finding the root word, or even another approach to tackle the problem is definitely worth developing if I want to improve the result

**(2) More words to ignore** 

There are many other words that may worth being considered to remove when locating the root word. But since they are not as easy as `THE` to be determined as meaningless, I did not put them in the list of words to ignore

Here are three examples I came across when glancing through my results

1. `NEW`
    - NEW FLYER INDUSTRIES CANADA ULC
    - NEW PIG
    - NEW RELIC INC (IOS)
    - ...
2. `NICE`
    - NICE ACTIVE VOIP
    - NICE PERFORM NPX
    - NICE VISION  NET 2.5
    - ...
3. `NORTH`    
    - NORTH SHORE INDUST'L SUPPLY
    - NORTH STAR
    - NORTH STATES INDUSTRIES
    - ...

**(3) Add mapping between common abbreviations**

I also notice that there are also some titles that are basically the same but some use abbreviations and some do not. Therefore, I think including a mapping between common abbreviations, such as `{'ASSOCIATION': ['ASSOC.','ASSN.']}`, will be helpful with correcting some wrong unifications.

For example, two titles below should have the same unified title but due to different use of abbreviations, I failed to produce the optimal results for them.

- CENTRAL ASSOC. FOR THE BLIND
- CENTRAL ASSOCIATION F/T BLIND

**(4) Keep original special characters in the unitied title**

When I was cleaning the raw title, I replaced all punctuations with single whitespace for simplification. However, some special characters may be wanted to remain in the unified title.

For example, the dash character may be wanted in unified title like `PORTO-POWER`, but my result for that unified title is `PORTO POWER`


### Run the codes below to produce the result

In [1]:
# Import library
import pandas as pd
import numpy as np
import string
import re

In [2]:
def main():
    '''
    The main function to perform the title unification including:
    1. Load the data from 'raw_titles.xlsx' in current directory
    2. Perform unifications for titles with the same starting word after cleaning
    3. Write the original raw titles and unified titles together to an excel
    '''
    
    # Load the data
    data = pd.read_excel('raw_titles.xlsx')
    
    # Sort the dataframe by raw titles alphabetically
    # Titles with same root word are put next to each other
    data.sort_values('raw_title', inplace=True)
    # Reset the index for dataframe and keep the original index
    data.reset_index(inplace = True)
    
    # Unify raw titles that start with the same word
    unified_data = unify_title(dataframe = data, 
                               raw_column = 'raw_title', 
                               unified_column = 'unified_title',
                               ignore = ['THE'],
                               remove = ['INC','PC','SC','GP','LP','LLP','CORP'
                                         'LLLP','LLC','LC','LTD','PLLC'])
    
    # Recover the original index
    unified_data.set_index('index', inplace=True)
    # Sort the dataframe by original index, so it looks the same
    unified_data.sort_index(inplace = True)
    
    # Write the dataframe to excel
    dataframe_to_excel(dataframe = unified_data, 
                       filename = 'unified_titles.xlsx',
                       sheetname = 'unified', 
                       with_index = False)

In [3]:
def dataframe_to_excel(dataframe, filename, sheetname, with_index = False):
    '''
    Write a pandas dataframe to excel
    
    Parameters
    ----------
    dataframe: pandas.core.frame.DataFrame
        Dataframe object that need to be written to excel
    filename: str
        Name of output excel file
    sheetname:
        Name of output sheet in excel
    with_index: bool
        Whether to include dataframe index as a column in excel or not
    
    Raises
    ------
    TypeError
        If input variables are not in correct data type
    '''
    
    # Validate the input variable type
    if not isinstance(dataframe, pd.DataFrame):
        raise TypeError("Input table must be pandas DataFrame")
    if not isinstance(filename, str):
        raise TypeError("Name of output excel file must be a string")
    if not isinstance(sheetname, str):
        raise TypeError("Name of output sheet in excel must be a string")
    if not isinstance(with_index, bool):
        raise TypeError("with_index can only take boolean value")
    
    # Initialize an Excel writier
    writer = pd.ExcelWriter(filename)
    # Write out the unified result
    dataframe.to_excel(writer, sheet_name = sheetname, index = with_index)
    writer.save()

In [4]:
def replace_punctuations(s, delimiter = ' '):
    '''
    Replace every punctuation character in the input string with delimiter
    
    Parameters
    ----------
    s: str
        Input string
    delimiter: str
        Delimiter used to replace punctuation character (the default is ' ')
        
    Returns
    -------
    new_s: str
        Return the string where every original punctuation character is replaced with delimiter
        
    Raises
    ------
    TypeError
        If the input is not a string
    '''
    
    # Validate the input variable type
    if not isinstance(s, str):
        raise TypeError("Input must be a string")
    if not isinstance(delimiter, str):
        raise TypeError("Delimiter must be a string")
        
    # Get all punctuations except period(.), which may be used in acronyms
    punctuations = ''.join([punct for punct in string.punctuation if punct != '.'])
    # Create the regex format to capture any punctuation characters
    regex_punct = r'['+re.escape(punctuations)+']'
    # Replace every punctuation character in the input string with a single space
    new_s = re.sub(regex_punct, delimiter, s)
    # Remove redundant spaces
    new_s = re.sub(r''+re.escape(delimiter)+'+',' ', new_s)
    # Return
    return new_s

In [5]:
def remove_words(title, wordList):
    '''
    Remove meaningless words from the title. 
    Mainly abbreviations for the form of business enterprise.
    
    - Coporations:
        - Corp: Corporation
        - INC: Incorporated Company
        - PC: Professional Coporation
        - SC: Service Coporation
    - Partnerships:
        - GP: Genearl Partnership
        - LP: Limited Partnership
        - LLP: Limited Liability Partnership
        - LLLP: Limited Liability Limied Partnership
    - Limited Liability Companies:
        - LLC: Limited Liability Company
        - LC: Limited Liability Company
        - LTD.: Limited Company
        - PLLC: Professional Limited Liability Company
    
    Parameters
    ----------
    title:str
        Input title
    wordList:[str]
        A list of words to remove from the title
    
    Returns
    -------
    clean_title: str
         Cleaned title after words removal
    
    Raises
    ------
    TypeError
        If the input word list for removal are not a list of strings
    '''
    # Validate input variable type
    if not isinstance(wordList, list):
        raise TypeError('The input word list for removal must be a list')
    if not all([isinstance(word, str) for word in wordList]):
        raise TypeError('Every word in the list must be in string format')    
    
    # Remove words from the title if it appears in the given word list for removal
    clean_title = [word for word in title.split() if word not in wordList]
    # Combine the rest of words together in a string separated by whitespace
    clean_title = ' '.join(clean_title)
    
    # Return the cleaned title
    return clean_title

In [6]:
def clean_title(title, wordList):
    '''
    Clean the input title and return the new one
    
    Parameters
    ----------
    wordList: [str]
        The list of words targeted to be removed from titles
        
    Returns
    -------
    new_title: str
        The cleaned title
    '''
    
    # Replace puncatuations in the title with whitespace
    new_title = replace_punctuations(title)
    # Remove words in the given list from the title
    new_title = remove_words(new_title, wordList)
    
    # If after the cleaning, nothing is left
    if new_title == '':
        # Skip the cleaning procedure and restore the original raw title
        new_title = title
    
    # Return the cleaned title
    return new_title

In [7]:
def root_word(title, ignore):
    '''
    Obtain the root word of a title based on the following assumptions:
    (1) The root word is at the beginning of the title
    (2) The root word is consisted of more than one character
    
    Parameters
    ----------
    title: str
        Input title with space as delimiter
    ignore: [str]
        The list of words to ignore when locating the root word of a title
        
    Returns
    -------
    root: str
        The root word of the input title
    
    Raises
    ------
    TypeError
        If the input title is not a string
    '''
    
    # Obtain the list of words in the title
    words = title.split()
    
    # Remove the first word from consideration if it is among the ignore words
    if words[0] in ignore:
        words.pop(0)
    
    # Assume the first word is root word
    root = words.pop(0)
    
    # If the first word has only one character
    if len(root) == 1:
        # Loop through the rest words in the list in order one by one, 
        # stop when reach a word that has more than one characters
        while len(words[0]) <= 1:
            # Pop out the first word from the list
            # Add the first word to root word with space delimiter
            root += ' ' + words.pop(0)

    # Return the root word
    return root

In [8]:
def longest_common_substring(s1, s2):
    '''
    Use dynamic programming to find the longest common substring, with complete words, 
    of two input strings
    
    Parameters
    ----------
    s1: str
        First input string
    s2: str
        Second input string
    
    Returns
    -------
    substring: str
        Return the longest common substring
        
    Raises
    ------
    TypeError
        If the inputs are not strings
    '''
    
    # Validate the input variable type
    if not isinstance(s1, str):
        raise TypeError("First input must be a string")
    if not isinstance(s2, str):
        raise TypeError("Second input must be a string")
    
    # Split the first string into list of words
    s1 = s1.split()
    # Split the second string into list of words
    s2 = s2.split()
    
    # Initialize a matrix M filled with zeroes to store 
    # lengths of the longest common substring
    # M[i][j] contains the lengh of longest common substring between 
    # s1[0,1,...,i-1] and s2[0,1,...,j-1]
    M = np.array([[0] * (1 + len(s2)) for i in range(1 + len(s1))])
    
    # Initialize the lenghth of the longest common substring to zero
    longest_length = 0
    
    # Compare all characters in s1 and all characters in s2 pair by pair
    for i in range(1, 1 + len(s1)):
        for j in range(1, 1 + len(s2)):
            # If i-1 th character in s1 matches j-1 th character in s2
            if s1[i - 1] == s2[j - 1]:
                # Add the longest length by 1
                M[i][j] = M[i - 1][j - 1] + 1
                # Update the longest_length
                longest_length = max(longest_length,M[i][j])
    
    # Find the end position of the longest common substring in s1
    s1_end = np.where(M == longest_length)[0].item(0)
    # Calculate the start position of the the longest common substring in s1
    s1_start = s1_end - longest_length
    # Extract the longest common substring
    substring = ' '.join(s1[s1_start: s1_end])
    
    # Return the longest common substring
    return substring

In [9]:
def unify_title(dataframe, raw_column, unified_column, ignore, remove):
    '''
    Unify similar titles in raw_column of dataframe and 
    put the same unified title for all of them in corresponding unified_column
    
    Parameters
    ----------
    dataframe: pandas.core.frame.DataFrame
        Input table containing at least a raw title column which need to be unified
    raw_column: str
        Column name in the dataframe that contains raw titles
    unified_column:str
        Column name in the dataframe that will store the unified titles
    ignore: [str]
        The list of words to ignore when locating the root word of a title
    remove: [str]
        The list of words targeted to be removed from titles
    
    Returns
    -------
    result: pandas.core.frame.DataFrame
        Return a table with both original raw titles and their unified titles
        
    Raises
    ------
    TypeError
        If the input table is not pandas dataframe, or column names are not strings
    '''
    
    # Validate the input variable type
    if not isinstance(dataframe, pd.DataFrame):
        raise TypeError("Input table must be pandas DataFrame")
    if not isinstance(raw_column, str):
        raise TypeError("Column name in the dataframe for raw titles must be a string")
    if not isinstance(unified_column, str):
        raise TypeError("Column name in the dataframe to store unified titles must be a string")
    
    # Make a deep copy of original data
    result = dataframe.copy()
    
    # Obtain the list of raw titles
    raw_title = result[raw_column]
    
    # Initialize i and j
    # i: the index of current raw title
    # j: the index of the raw title to compare with
    i = 0
    j = i
    # Obtain the max value for index j (The number of rows in dataframe)
    _max = result.shape[0]
    
    # While the index j is NOT out of range
    while j < _max:
        # Obtain the current raw title and clean it
        raw = clean_title(raw_title[i], remove)
        # Obtain the root word from the raw_title[i]
        root = root_word(raw, ignore)
        # Create an empty list to store longest common substring for each pair of raw titles
        common_title_list = []

        # Loop through all raw titles starting from index i
        # Find the longest common substring between raw_title[i] and raw_title[j] 
        # Stop when
        # (1) Reach the last row (j == _max - 1)
        # (2) raw_title[i] and raw_title[j] DO NOT have the same root word (Should NOT be unified)
        for j in range(i, _max):
            # Obtain the current raw title and clean it
            new_raw = clean_title(raw_title[j], remove)
            # Obtain the root word for raw_title[j]
            new_root = root_word(new_raw, ignore)
            
            # If the two raw titles have the same root word
            # Try to unify these two titles
            if new_root == root:
                # Find the longest common substring
                common_title = longest_common_substring(raw,new_raw)
                # Add the current longest common substring to the list
                common_title_list.append(common_title)
                # If the index reach the last row
                if j == _max - 1:
                    # Use the shortest substring from the common_title_list
                    # to be the unified title
                    unified_title = min(common_title_list, key = len)
                    # If the unified title does not contain the root word
                    if root not in unified_title:
                        # Discard the longest common substring and
                        # set the root word to be the unified title
                        unified_title = root
                    
                    # Insert the same unified title for all current raw titles
                    result.loc[i:j,unified_column] = unified_title
                    # Return the new dataframe with filled unified_column
                    return result
            
            # If the two raw titles DO NOT have the same root word
            # Finish the current title unification
            else:
                # Use the shortest substring from the common_title_list
                # to be the unified title
                unified_title = min(common_title_list, key = len)
                # If the unified title does not contain the root word
                if root not in unified_title:
                    # Discard the longest common substring and
                    # set the root word to be the unified title
                    unified_title = root
                    
                # Insert the same unified title for all current raw titles
                result.loc[i:j-1,unified_column] = unified_title
                # Reset index i to start title unification for the next raw title
                i = j
                # Jump out of the inner j for loop
                break

In [10]:
# Execute the main() function
main()